In [30]:
import vk_api
import datetime
import pytz
import json
import openpyxl
import gspread
from oauth2client.service_account import ServiceAccountCredentials

tz = pytz.timezone('Europe/Moscow')

def read_tokens_from_excel(filename, sheet_title, sheet_range_tokens):
    try:
        workbook = openpyxl.load_workbook(filename)
        sheet = workbook[sheet_title]
        tokens = [cell.value for row in sheet[sheet_range_tokens] for cell in row if cell.value]
        return tokens
    except Exception as e:
        print("Произошла ошибка при чтении файла Excel: ", e)
        exit(1)

def write_data_to_google_sheet(data, spreadsheet_key, group_id):
    try:
        scope = ['https://spreadsheets.google.com/feeds',
                 'https://www.googleapis.com/auth/drive']
        credentials = ServiceAccountCredentials.from_json_keyfile_name(
            '../../../access/credentials.json', scope)
        client = gspread.authorize(credentials)
        spreadsheet = client.open_by_key(spreadsheet_key)

        sheet_title = f"{group_id}"  # Название листа, содержащего ID сообщества

        try:
            sheet = spreadsheet.worksheet(sheet_title)
        except gspread.exceptions.WorksheetNotFound:
            sheet = spreadsheet.add_worksheet(title=sheet_title, rows="100", cols="20")
            sheet.append_row(['User ID', 'Date', 'Timezone'])

        sheet.clear()
        sheet.append_rows(data)
        print("Данные успешно записаны в Google Таблицу.")
    except Exception as e:
        print("Произошла ошибка при записи данных в Google Таблицу: ", e)
        exit(1)

def get_first_message_date(vk, user_id):
    history = vk.messages.getHistory(user_id=user_id, rev=1)
    first_contact = history['items'][0]['date']
    return first_contact

def get_date_in_timezone(timestamp, timezone):
    date = datetime.datetime.fromtimestamp(timestamp, tz=pytz.utc).astimezone(timezone)
    return date.strftime('%Y-%m-%d, %H:%M:%S, %Z%z')

# Чтение конфигурации из файла
with open('../../../access/config.json') as config_file:
    config = json.load(config_file)

filename = '../../../access/db.xlsx'
sheet_title = config['db_sheet_title']
sheet_range_tokens = config['db_sheet_range_tokens']
sheet_range_group = config['db_sheet_range_group']
spreadsheet_key = config['target_spreadsheet_key']

# Чтение списка токенов из файла Excel
tokens = read_tokens_from_excel(filename, sheet_title, sheet_range_tokens)

# Создание сессии API ВК
vk_sessions = [vk_api.VkApi(token=token) for token in tokens]

# Обработка данных для каждого токена
for vk_session in vk_sessions:
    vk = vk_session.get_api()

    # Сбор данных
    data = []
    dialogs = vk.messages.getConversations()
    messages_count = dialogs['count']
    print(f'В группе {group_id} {messages_count} сообщений')

    count = 200
    offset = 0

    while offset < messages_count:
        dialogs = vk.messages.getConversations(count=count, offset=offset)['items']
        all_interlocutors = [user['conversation']['peer']['id'] for user in dialogs]

        for interlocutor in all_interlocutors:
            first_message_date = get_first_message_date(vk, interlocutor)
            formatted_date = get_date_in_timezone(first_message_date, tz)
            data.append([interlocutor, formatted_date])

        offset += count

    # Получение ID сообщества из токена
    group_id = vk.groups.getById()[0]['id']

    # Запись данных в Google Таблицу
    write_data_to_google_sheet(data, spreadsheet_key, group_id)


В группе 40554507 62 сообщений
